### 1️⃣ Importaciones y configuración

In [ ]:
import sys
import os

BASE_DIR = os.path.abspath(os.path.join(os.path.dirname("__file__"), "..", ".."))
OUTPUT_DIR = os.path.join(BASE_DIR, "output")

TRAIN_CSV = os.path.join(OUTPUT_DIR, "train_processed.csv")
VAL_CSV   = os.path.join(OUTPUT_DIR, "val_processed.csv")
TEST_CSV  = os.path.join(OUTPUT_DIR, "test_processed.csv")

sys.path.insert(0, BASE_DIR)

from src.part_2.main import run_training
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
from sklearn.neural_network import MLPRegressor
import pandas as pd
from sklearn.linear_model import LinearRegression


### 2️⃣ Funciones de evaluación de métricas

In [ ]:
def mape(y_true, y_pred):
    """Mean Absolute Percentage Error"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mape_val = mape(y_true, y_pred)
    return {
        "MSE": float(mse),
        "MAE": float(mae),
        "MAPE": float(mape_val)
    }


### 3️⃣ Funciones de entrenamiento de los modelos BP-F y MLR-F

In [20]:
def train_bp_f(X_train, y_train, X_test, activation="identity"):
    """
    BP-F model using scikit-learn's MLPRegressor
    """
    mlp = MLPRegressor(
        hidden_layer_sizes=(20, 10),
        activation=activation,
        solver="adam",
        max_iter=200,
        tol=1e-4,
        random_state=42
    )
    mlp.fit(X_train, y_train)
    preds = mlp.predict(X_test)
    return preds, mlp

def train_mlr_f(X_train, y_train, X_test):
    """
    MLR-F model using scikit-learn's LinearRegression
    """
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    preds = lr.predict(X_test)
    return preds, lr


### 4️⃣ Cargar datos usando las variables del notebook


In [18]:
df_train = pd.read_csv(TRAIN_CSV)
df_test  = pd.read_csv(TEST_CSV)

# Separar características y target
X_train = df_train.drop(columns=["SalePrice", "PID"])
y_train = df_train["SalePrice"]

X_test  = df_test.drop(columns=["SalePrice", "PID"])
y_test  = df_test["SalePrice"]


### 5️⃣ Entrenamiento y predicción de los modelos

In [ ]:
# BP Custom Neural Network
activation_bp = "sigmoid"  # Desired activation function

nn, pid_test, y_test_values, preds_bp, trainer = run_training(activation_bp, verbose=False, CSV_TRAIN=TRAIN_CSV, CSV_VAL=VAL_CSV, CSV_TEST=TEST_CSV)

print("BP Custom:")
evaluate_model(y_test_values, preds_bp)


Training Neural Network...


/Users/miguelangelcabanillas/Desktop/Neural and Evolutionary Computation/Activity-1/src/part_2/neural_net.py:77: RuntimeWarning: divide by zero encountered in matmul
  self.h[l] = self.w[l] @ self.xi[l-1] - self.theta[l]
/Users/miguelangelcabanillas/Desktop/Neural and Evolutionary Computation/Activity-1/src/part_2/neural_net.py:77: RuntimeWarning: overflow encountered in matmul
  self.h[l] = self.w[l] @ self.xi[l-1] - self.theta[l]
/Users/miguelangelcabanillas/Desktop/Neural and Evolutionary Computation/Activity-1/src/part_2/neural_net.py:77: RuntimeWarning: invalid value encountered in matmul
  self.h[l] = self.w[l] @ self.xi[l-1] - self.theta[l]


BP Custom:


{'MSE': 4636042018.56689, 'MAE': 50669.05423284154, 'MAPE': 28.513481605696605}

In [ ]:
# BP-F Model
activation_map_bp_f = {
    "linear": "identity",
    "sigmoid": "logistic",
    "tanh": "tanh",
    "relu": "relu"
}

activation_bp_f = activation_map_bp_f.get(activation_bp)

preds_bp_f, bp_f_model = train_bp_f(X_train, y_train, X_test, activation_bp_f)

print("BP-F:")
evaluate_model(y_test, preds_bp_f)


/opt/homebrew/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/opt/homebrew/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/opt/homebrew/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


BP-F:


/opt/homebrew/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/homebrew/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/opt/homebrew/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/opt/homebrew/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


{'MSE': 676442429.7235671,
 'MAE': 18231.168793909917,
 'MAPE': 11.08822532026241}

In [ ]:
# MLR-F Model
preds_mlr_f, mlr_model = train_mlr_f(X_train, y_train, X_test)

print("MLR-F:")
evaluate_model(y_test, preds_mlr_f)

In [ ]:
# Comparative Metrics
metrics = pd.DataFrame({
    "BP Custom": evaluate_model(y_test_values, preds_bp),
    "BP-F": evaluate_model(y_test, preds_bp_f),
    "MLR-F": evaluate_model(y_test, preds_mlr_f)
}).T

display(metrics)
